<a href="https://colab.research.google.com/github/samuelhtampubolon/SDPM2025/blob/main/Pinjaman_Bank_C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

# Langkah 1: Baca file
file_path = '/content/sample_data/Pinjaman_Bank_C.xlsx'
df = pd.read_excel(file_path)

In [3]:
import argparse, re, pandas as pd, numpy as np
#argparse is a built-in Python module that facilitates the creation of user-friendly command-line interfaces (CLIs) for Python programs

def strip_all_obj(df):
    for c in df.select_dtypes(include=["object"]).columns:
        df[c] = df[c].astype(str).str.strip()
        df[c] = df[c].replace({"None": np.nan, "": np.nan, "nan": np.nan, "NaN": np.nan, "N/A": np.nan, "—": np.nan})
    return df

def parse_currency(s):
    if pd.isna(s): return np.nan
    s = str(s).strip().lower()
    if s in {"", "na", "n/a", "nan", "—"}: return np.nan
    m = re.search(r'([\d]+(?:[\.,]\d+)?)\s*(m|jt|juta)\b', s)  # IDR 1.5m / 1,5 jt
    if m:
        val = float(m.group(1).replace(",", "."))
        return int(round(val * 1_000_000))
    s2 = re.sub(r'[^\d\.,-]', '', s)
    if s2.count(",") > 1 and "." not in s2: s2 = s2.replace(",", "")
    if s2.count(".") > 1 and "," not in s2: s2 = s2.replace(".", "")
    s2 = s2.replace(".", "").replace(",", "")
    try: return int(float(s2))
    except: return np.nan

def parse_rate(x):
    if pd.isna(x): return np.nan
    s = str(x).strip().lower().replace(" ", "")
    if s in {"tbd", "—", "na", "n/a"}: return np.nan
    if s.endswith("%"): s = s[:-1]
    s = s.replace(",", ".")
    try:
        val = float(s)
        return round(val*100, 4) if val < 1 else round(val, 4)  # 0.12 -> 12%
    except: return np.nan

def parse_tenure(x):
    if pd.isna(x): return np.nan
    s = str(x).strip().lower()
    s = s.replace("months","m").replace("month","m").replace("bln","m")
    s = s.replace("tahun","yr").replace("thn","yr").replace("years","yr").replace("year","yr").replace(" ", "")
    if s in {"", "nan"}: return np.nan
    m = re.search(r'([\d\.]+)yr', s)
    if m:
        try: return int(round(float(m.group(1))*12))
        except: return np.nan
    m2 = re.search(r'(\d+)m$', s)
    if m2: return int(m2.group(1))
    try: return int(float(s))
    except: return np.nan

def normalize_branch(s):
    return np.nan if pd.isna(s) else str(s).strip().title()

def normalize_employment(s):
    if pd.isna(s): return np.nan
    s = str(s).strip().lower().replace("-", " ")
    mapping = {"full time":"Full-time","full  time":"Full-time","fulltime":"Full-time","contract":"Contract",
               "self employed":"Self-employed","self  employed":"Self-employed","unemployed":"Unemployed","student":"Student"}
    return mapping.get(s, s.title())

def normalize_purpose(s):
    if pd.isna(s): return np.nan
    s = str(s).strip().lower()
    mapping = {"rumah":"Home","mobil":"Car","sekolah":"Education","usaha":"Business",
               "renovasi":"Renovation","hp":"Gadget","liburan":"Travel"}
    return mapping.get(s, s.title())

def normalize_approval(s):
    if pd.isna(s): return np.nan
    s = str(s).strip().lower()
    if s in {"on hold","pending docs","pending"}: return "Pending"
    if s.startswith("approved"): return "Approved"
    if s == "rejected": return "Rejected"
    return s.title()

def normalize_loan_id(s):
    if pd.isna(s) or str(s).strip() == "": return np.nan
    s = str(s).strip().upper().replace("LOAN","LN").replace("_","-")
    m = re.search(r'(LN)-?(\d{4})-?(\d{5,})', s)
    return f"LN-{m.group(2)}-{int(m.group(3)):05d}" if m else s

def normalize_nik(s):
    if pd.isna(s): return np.nan
    digits = re.sub(r'\D','', str(s))
    return digits if len(digits)==16 else np.nan

def compute_emi(principal, annual_rate_pct, tenure_months):
    try:
        p = float(principal); r = float(annual_rate_pct)/100/12; n = int(tenure_months)
        if p<=0 or n<=0 or r<0: return np.nan
        if r==0: return round(p/n, 2)
        return round(p*r*(1+r)**n/((1+r)**n-1), 2)
    except: return np.nan

def main(i_path, o_path):
    df = pd.read_excel(i_path, sheet_name=0, dtype=str)
    df = strip_all_obj(df)
    df["Loan_ID"] = df["Loan_ID"].apply(normalize_loan_id)
    df["National_ID"] = df["National_ID"].apply(normalize_nik)
    df["Application_Date"] = pd.to_datetime(df["Application_Date"], errors="coerce", dayfirst=True, infer_datetime_format=True)
    df["Loan_Amount_IDR"] = df["Loan_Amount_IDR"].apply(parse_currency)
    df["Monthly_Income_IDR"] = df["Monthly_Income_IDR"].apply(parse_currency)
    df["Interest_Rate"] = df["Interest_Rate"].apply(parse_rate)
    df["Tenure_Months"] = df["Tenure"].apply(parse_tenure)
    df["Branch"] = df["Branch"].apply(normalize_branch)
    df["Employment_Status"] = df["Employment_Status"].apply(normalize_employment)
    df["Loan_Purpose"] = df["Loan_Purpose"].apply(normalize_purpose)
    df["Approval_Status"] = df["Approval_Status"].apply(normalize_approval)

    # deduplicate by Loan_ID (if exists)
    df = df.sort_values(["Loan_ID","Application_Date"]).drop_duplicates(subset=["Loan_ID"], keep="last")

    # cap implausible amounts/income; set invalid to NaN
    df["Loan_Amount_IDR"] = df["Loan_Amount_IDR"].apply(lambda x: np.nan if (pd.notna(x) and (x < 1_000_000 or x > 5_000_000_000)) else x)
    df["Monthly_Income_IDR"] = df["Monthly_Income_IDR"].apply(lambda x: np.nan if (pd.notna(x) and x <= 0) else x)

    df["DTI"] = np.where((df["Loan_Amount_IDR"].notna()) & (df["Tenure_Months"].notna()) & (df["Monthly_Income_IDR"].notna()),
                         (df["Loan_Amount_IDR"]/df["Tenure_Months"])/df["Monthly_Income_IDR"], np.nan)
    df["EMI_Est"] = df.apply(lambda r: compute_emi(r["Loan_Amount_IDR"], r["Interest_Rate"], r["Tenure_Months"]), axis=1)

    cols = ["Loan_ID","Customer_Name","National_ID","Application_Date","Loan_Amount_IDR","Interest_Rate",
            "Tenure_Months","Monthly_Income_IDR","Employment_Status","Loan_Purpose","Branch","Approval_Status","DTI","EMI_Est"]
    df[cols].to_excel(o_path, index=False, sheet_name="clean_loans")
    print("Saved:", o_path)

# if __name__ == "__main__":
#     import sys, argparse
#     p = argparse.ArgumentParser()
#     p.add_argument("--input","-i", required=True)
#     p.add_argument("--output","-o", default="loan_data_clean.xlsx")
#     a = p.parse_args()
#     main(a.input, a.output)

# Call the main function directly with hardcoded paths for Colab
main('/content/sample_data/Pinjaman_Bank_C.xlsx', 'loan_data_clean.xlsx')

Saved: loan_data_clean.xlsx


/tmp/ipython-input-2259976943.py:99: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df["Application_Date"] = pd.to_datetime(df["Application_Date"], errors="coerce", dayfirst=True, infer_datetime_format=True)
/tmp/ipython-input-2259976943.py:99: UserWarning: Parsing dates in %Y-%m-%d %H:%M:%S format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df["Application_Date"] = pd.to_datetime(df["Application_Date"], errors="coerce", dayfirst=True, infer_datetime_format=True)
